Algos in Python
================

This document covers how to instantiate an abstract algorithm, and then configure it with a config block

Instantiating an abstract algo in Python is very similar to instantiating an algo in C++.

A note about providing a custom algo/arrow implementation, 


The first step in this process is to actually provide an instantiation for the given abstract algo.
In this tutorial we assume that the selected algo is one provided by the existing Kwiver Python bindings.
Implementing an abstract algo in Python and then instantiating it is slightly more complex and covered in a later piece of documentation.


Below we have selected the Image IO C++ algo to instantiate

In [ ]:
from kwiver.vital.algo import ImageIO

class CustomImageIOImpl(ImageIO):
    """
    Custom implementation of the Abstract Kwiver Vital algo ImageIO
    """
    def __init__(self):
        # THE FOLLOWING LINE IS REQUIRED BY PYBIND11
        super.__init__(self)

Notice that we subclass the algorithm that we wish to instantiate. Additionally, note that we call the initialization function for the parent (the desired algo) class. This is required by PyBind11 to ensure proper inheritance across languages.

Next we expose the algorithm for registration

In [ ]:
def __vital_algorithm_register__():
    from kwiver.vital.algo import algorithm_factory as af
    
    # This name is the name of the class you used to instantiate the algo
    # the class name can be anything, but this string needs to mirror that
    inst_name = "CustomImageIOImpl"
    # determine if algo is already loaded
    if af.has_algorithm_impl_name(
        CustomImageIOImpl.static_type_name(),
        inst_name
    ):
        return
    af.add_algorithm( inst_name,
                     "kwiver.vital.algo.ImageIO",
                     CustomImageIOImpl,
                     )
    af.mark_algorithm_as_loaded( inst_name )

This registration block is very similar to the process for registering a C++ algo inst during runtime. Here we leverage python's special treatment of dunder (or magic if you prefer) methods to inform Kwiver of this algo instantiation. 

Note that at this point, our algo implementation, while useable, is still abstract as we have not provided concrete implementations of any of the exposed methods.

Additionally, we need to expose and implement the ability to configure the algo
with Kwiver config blocks. These are supported in Python via bindings that expose the config block under `kwiver.vital.config`

Our class then, fully implemented might look something like this.

In [ ]:
from kwiver.vital.algo import ImageIO
import PIL

class CustomImageIOImpl(ImageIO):
    """
    Custom implementation of the Abstract Kwiver Vital algo ImageIO
    """
    def __init__(self):
        super.__init__(self)
        # initialize any member attributes
    
    def get_configuration(self):
        return super().get_configuration()

    def set_configuration(self, conf_c = None):
        cnf = self.get_configuration()
        cnf.merge_config(cnf, conf_c)
    
    def load(im):
        return PIL.Image.open(im)
        
    def save(im):
        im.save("PIL_im.png")



Arrows/algos in Python can be executed by a pipeline via a Sprokit Process, however they can also be called directly from Python code.

The example below demonstrates this style of useage. This is of course, a very basic example, but it demonstrates enough to implement a custom algo and execute it via python.

In [ ]:
from kwiver.vital import types as kvt
from kwiver.vital import algo
im_dir = "./shapes.png"

PIL_io = algo.ImageIO.create("CustomImageIOImpl")
im = PIL_io.load(im_dir)
# operations can now be performed on the image as if it were a typical OCV python image, because it is. This algo is basically just a very thin wrapper around some basic opencv functionality
kvt.BoundingBoxD(0,10,0,25)

# alternatively all this can be done via the algorithm factory

import kwiver.vital.algo.algorithm_factory as af
PIL_io2 = af.create_algorithm("ImageIO","CustomImageIOImpl")

Algorithms can be instantiated and configured programmatically during runtime via configurations (as opposed to Sprokit Processes).

The following examples shows how the algorithm implemented above would be programmatically instantiated and configured

In [ ]:
from kwiver.vital.config import config
from kwiver.vital import algo
        

# First we need to get a reference to the base algo type we want to instantiate
PIL_io = algo.ImageIO

# Next we must load a config_block object
# Currently this cannot be done from file in Python
# But we can construct a config block programmatically
# So lets assume we've done that

io_conf = load_config(conf_dir)
if algo.ImageIO.check_nested_algo_configuration("IO",io_conf):
     algo.ImageIO.set_nested_algo_configuration("IO",io_conf, PIL_io)
else:
    raise RuntimeError("Unable to properly load conf")

# Now PIL_io holds our instantiated algorithm
# Ready to go and configured by a conf object
# The config syntax does not change between c++ and Python


C++ arrow from Python

Arrows currently existing with a C++ implementation can be accessed and utilized directly from Python code with no need for any extra steps.
The process is, much like above, simply loading in the arrow for use, almost exactly the way it's done by C++

In [ ]:
from kwiver.vital.algos import algorithm_factory as af
from kwiver.vital.config import config

uuid_mod = af.create_algorithm("uuid","uuid")
uuid = uuid.create_uuid()
print(uuid)

The last component of an algo/arrow Python interface is creating an abstract algorithm from python and exposing it to the Kwiver plugin loader.

In [ ]:
from kwiver.vital.algo import algorithm_factory as af, _algorithm
from kwiver.vital import types as kvt

class PyImageIO(_algorithm):
    @static
    def create(name: str) -> PyImageIO:
        pass
    @static
    def registered_names() -> list:
        pass
    def type_name() -> str:
        pass
    @static
    def get_nested_algo_configuration() -> None:
        pass